In [1]:
using JSON

In [2]:
detData = JSON.parsefile("detCleaned.json")

Dict{String,Any} with 30 entries:
  "54" => Dict{String,Any}(Pair{String,Any}("name", "GD79B"),Pair{String,Any}("…
  "58" => Dict{String,Any}(Pair{String,Any}("name", "GD89C"),Pair{String,Any}("…
  "65" => Dict{String,Any}(Pair{String,Any}("name", "GD00B"),Pair{String,Any}("…
  "51" => Dict{String,Any}(Pair{String,Any}("name", "GD61C"),Pair{String,Any}("…
  "53" => Dict{String,Any}(Pair{String,Any}("name", "GD76C"),Pair{String,Any}("…
  "49" => Dict{String,Any}(Pair{String,Any}("name", "GD61A"),Pair{String,Any}("…
  "59" => Dict{String,Any}(Pair{String,Any}("name", "GD89D"),Pair{String,Any}("…
  "61" => Dict{String,Any}(Pair{String,Any}("name", "GD91B"),Pair{String,Any}("…
  "67" => Dict{String,Any}(Pair{String,Any}("name", "GD00D"),Pair{String,Any}("…
  "42" => Dict{String,Any}(Pair{String,Any}("name", "GD32A"),Pair{String,Any}("…
  "62" => Dict{String,Any}(Pair{String,Any}("name", "GD91C"),Pair{String,Any}("…
  "43" => Dict{String,Any}(Pair{String,Any}("name", "GD32B"),Pair{String,An

In [3]:
detMass = JSON.parsefile("detectorData_gerda-p2.json")["detData"]

Dict{String,Any} with 40 entries:
  "24" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "4"  => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "1"  => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "12" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "29" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "20" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "32" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "2"  => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "6"  => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "25" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "23" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,Any}(…
  "22" => Dict{String,Any}(Pair{String,Any}("massErr", 0.001),Pair{String,A

In [4]:
function get_det_prop(name, prop)
    for k in keys(detMass)
        if detMass[k]["name"] == name
            return detMass[k][prop]
        end
    end
end

get_det_prop (generic function with 1 method)

In [5]:
function volume_cylinder(radius, height)
    pi*radius*radius*height
end

volume_cylinder (generic function with 1 method)

In [6]:
function volume_cone(radius, height)
    1./3. * volume_cylinder(radius, height)
end

volume_cone (generic function with 1 method)

In [7]:
function volume_cutcone(radius, cradius, cheight)
    1./3. * cheight * (radius*radius + radius*cradius + cradius*cradius)
end

volume_cutcone (generic function with 1 method)

In [8]:
function volume_BEGe(height, radius, gr_depth, gr_radius, gr_Radius, cheight, cradius)
    base   = volume_cylinder(radius, height-cheight)
    groove = volume_cylinder(gr_Radius, gr_depth) - volume_cylinder(gr_radius, gr_depth)
    cone   = volume_cutcone(radius, cradius, cheight)
    base - groove + cone
end

volume_BEGe (generic function with 1 method)

In [9]:
function volume_active(height, radius, gr_depth, gr_radius, gr_Radius, cheight, cradius, DL)
    volume = volume_BEGe(height, radius, gr_depth, gr_radius, gr_Radius, cheight, cradius)
    volume_DL = volume_BEGe(height, radius, 0, 0, 0, cheight, cradius)

    nheight  = height - 2*DL         # total height
    nradius  = radius - DL           # total radius
    
    if cheight > 0 && cradius > 0
        beta = atan(cheight/(radius-cradius))
        x    = DL/cos(beta)                    
        x_p  = DL*cheight/(radius-cradius)
        x_pp = x-x_p
        
        ncheight = cheight - DL + x_pp              # corner height (upper)
        ncradius = radius - DL - ncheight/tan(beta) # corner radius
        
        vol1 = volume_cylinder(nradius, nheight-ncheight)
        vol2 = volume_cylinder(gr_Radius, DL)
        vol3 = volume_cutcone(nradius, ncradius, ncheight)
        nvol = vol1 + vol2 + vol3
        
        #@printf "%.3f\n" vol2/1e3*5.56
        
        volume_DL -= nvol
    else
        volume_DL -= volume_cylinder(nradius, nheight) + volume_cylinder(gr_Radius, DL)
    end
    volume - volume_DL
end

volume_active (generic function with 1 method)

In [12]:
println( "name  : DL Am : DL Co : volume : f_Am  : f_Co  : df   : m_am : m_co : dma : dma\%" )
println( "***************************************************" )

df_tot = 0
dm_act_tot = 0
mass_tot = 0
c = 0

for k in keys(detData)
    d = detData[k]
    name = d["name"]
    h = d["height-mm"]
    r = d["radius-mm"]
    gr_d = d["groove"]["depth-mm"]
    gr_r = d["groove"]["inner-radius-mm"]
    gr_R = d["groove"]["outer-radius-mm"]
    ch = 0
    cr_rad = 0
    cr = 0
    DL_Am = d["dead-layer"]["top-thickness-nm"] / 1e6
    DL_Co = d["dl-co-mm"]
    
    mass  = get_det_prop(name, "mass")
    enr   = get_det_prop(name, "enrichment")
    amass = get_det_prop(name, "activemass")
    
    if get(d, "cone", 0) != 0
        ch = h - d["cone"]["height-mm"]
        cr = d["cone"]["radius-deg"]
    end
    
    volume = volume_BEGe(h,r,gr_d,gr_r,gr_R,ch,cr)/1e3
    volume_act = volume_active(h,r,gr_d,gr_r,gr_R,ch,cr, DL_Am)/1e3
    
    f_act = volume_act / volume
    m_act = mass * f_act
    
    volume_act_co = volume_active(h,r,gr_d,gr_r,gr_R,ch,cr, DL_Co)/1e3
    f_act_co = volume_act_co / volume
    m_act_co = mass * f_act_co
    
    df = f_act - f_act_co
    dm_act = m_act - m_act_co
    
    if name != "GD02D"
        df_tot += df
        dm_act_tot += dm_act
        mass_tot += m_act
        c += 1
    end
    
    @printf "%s : %.3f : %.3f : " name DL_Am DL_Co
    @printf "%6.2f : %.3f : %.3f : %.3f : " volume f_act f_act_co df 
    @printf "% .0f :  %.0f : %4.1f : %2.1f \n" m_act*1000 m_act_co*1000 dm_act*1000 dm_act*100/m_act
end
println( "*************************************" )
@printf "average active fraction difference %.1f pc \n" df_tot*100/c
@printf "average active mass difference %.1f pc\n" dm_act_tot*100/mass_tot
println( "*************************************" )

name  : DL Am : DL Co : volume : f_Am  : f_Co  : df   : m_am : m_co : dma : dma%
***************************************************
GD79B : 1.040 : 1.190 : 134.33 : 0.881 : 0.865 : 0.016 :  649 :  636 : 12.1 : 1.9 
GD89C : 0.990 : 1.210 : 108.13 : 0.875 : 0.848 : 0.027 :  520 :  505 : 15.8 : 3.0 
GD00B : 1.040 : 1.390 : 126.23 : 0.880 : 0.842 : 0.038 :  614 :  587 : 26.6 : 4.3 
GD61C : 0.920 : 1.090 : 115.15 : 0.888 : 0.868 : 0.020 :  563 :  550 : 12.6 : 2.2 
GD76C : 1.140 : 1.350 : 149.55 : 0.878 : 0.857 : 0.021 :  724 :  706 : 17.6 : 2.4 
GD61A : 1.010 : 1.410 :  88.55 : 0.887 : 0.845 : 0.042 :  648 :  618 : 31.0 : 4.8 
GD89D : 1.020 : 1.400 :  96.62 : 0.864 : 0.816 : 0.048 :  455 :  429 : 25.1 : 5.5 
GD91B : 0.960 : 1.160 : 118.05 : 0.889 : 0.867 : 0.022 :  578 :  563 : 14.4 : 2.5 
GD00D : 1.020 : 1.250 : 147.64 : 0.890 : 0.866 : 0.024 :  723 :  704 : 19.3 : 2.7 
GD32A : 0.910 : 1.130 :  37.97 : 0.849 : 0.815 : 0.034 :  389 :  373 : 15.8 : 4.1 
GD91C : 0.960 : 1.310 : 113.98 : 0.88